In [30]:
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics.pairwise import pairwise_distances_argmin
import warnings

In [31]:
def expand(a,b):
    d=(b-a) * 0.05
    return a-d,b+d

In [32]:
np.random.seed(0)
n=5
n_samples=1000
pi=np.random.rand(n)
pi /=pi.sum()
print('init pi : ',pi)

init pi :  [ 0.19356424  0.25224431  0.21259213  0.19217803  0.14942128]


In [33]:
A=np.random.rand(n,n)
mask=np.zeros((n,n),dtype=bool)
mask[0][1]=mask[0][4]=True
mask[1][0]=mask[1][2]=True
mask[2][1]=mask[2][3]=True
mask[3][2]=mask[3][4]=True
mask[4][0]=mask[4][3]=True


In [34]:
A[mask]=0

In [42]:
for i in range(n):
    A[i]/=A[i].sum()
    print('State Transform Matrix: \n',A)

State Transform Matrix: 
 [[ 0.25822029  0.          0.35651955  0.38526017  0.        ]
 [ 0.          0.34669639  0.          0.6067387   0.04656491]
 [ 0.04868208  0.          0.46521279  0.          0.48610513]
 [ 0.3825259   0.31237801  0.          0.30509609  0.        ]
 [ 0.          0.09539815  0.62865435  0.          0.2759475 ]]
State Transform Matrix: 
 [[ 0.25822029  0.          0.35651955  0.38526017  0.        ]
 [ 0.          0.34669639  0.          0.6067387   0.04656491]
 [ 0.04868208  0.          0.46521279  0.          0.48610513]
 [ 0.3825259   0.31237801  0.          0.30509609  0.        ]
 [ 0.          0.09539815  0.62865435  0.          0.2759475 ]]
State Transform Matrix: 
 [[ 0.25822029  0.          0.35651955  0.38526017  0.        ]
 [ 0.          0.34669639  0.          0.6067387   0.04656491]
 [ 0.04868208  0.          0.46521279  0.          0.48610513]
 [ 0.3825259   0.31237801  0.          0.30509609  0.        ]
 [ 0.          0.09539815  0.62865435 

In [36]:
means=np.array(((30,30),(0,50),(-25,30),(-15,0),(15,0)))
covars=np.empty((n,2,2))

In [43]:
for i in range(n):
    covars[i]=np.diag(np.random.rand(2)+0.001)*10
print('covs:',covars)

covs: [[[ 2.31742335  0.        ]
  [ 0.          7.65911699]]

 [[ 9.45123519  0.        ]
  [ 0.          7.50999249]]

 [[ 3.40403819  0.        ]
  [ 0.          4.90548937]]

 [[ 3.39985117  0.        ]
  [ 0.          1.80490261]]

 [[ 1.71986599  0.        ]
  [ 0.          4.64450977]]]


In [38]:
model=hmm.GaussianHMM(n_components=n,covariance_type='full')
model.startprob_=pi
model.transmat_=A
model.means_=means
model.covars_=covars
sample,labels=model.sample(n_samples,random_state=0)

In [39]:
sample[0]

array([-24.30917502,  31.84451547])

In [41]:
model=hmm.GaussianHMM(n_components=n,covariance_type='full',n_iter=10)
model.fit(sample)
y=model.predict(sample)
np.set_printoptions(suppress=True)
print('init prob :',model.startprob_)
print('Transfor Maxtrix : ',model.transmat_)
print('Mean : ',model.means_)
print('covs : ',model.covars_)


init prob : [ 1.  0.  0.  0.  0.]
Transfor Maxtrix :  [[ 0.47688564  0.48905109  0.03406326  0.          0.        ]
 [ 0.6294964   0.26978417  0.          0.          0.10071942]
 [ 0.43333333  0.          0.24444444  0.32222222  0.        ]
 [ 0.          0.          0.43902439  0.28455285  0.27642276]
 [ 0.          0.03092784  0.          0.60824742  0.36082474]]
Mean :  [[-25.03026589  30.10629378]
 [ 14.94392893  -0.11255187]
 [ 30.0639231   30.22329147]
 [-14.83607421   0.05082308]
 [ -0.60371631  49.96859881]]
covs :  [[[ 0.18351585  0.02577099]
  [ 0.02577099  5.45292363]]

 [[ 7.86488426 -0.16833308]
  [-0.16833308  6.31717184]]

 [[ 2.64868455  0.47269017]
  [ 0.47269017  8.5150928 ]]

 [[ 6.25312678 -0.49577678]
  [-0.49577678  6.21899877]]

 [[ 4.09533483  0.03786587]
  [ 0.03786587  4.48396995]]]


In [44]:
order =pairwise_distances_argmin(means,model.means_)
order

array([2, 4, 0, 3, 1], dtype=int64)

In [45]:
pi_hat=model.startprob_[order]
A_hat=model.transmat_[order]
A_hat=A_hat[:,order]
means_hat=model.means_[order]
covars_hat=model.covars_[order]
change=np.empty((n,n_samples),dtype=np.bool)
for i in range(n):
    change[i]=y==order[i]
for i in range(n):
    y[change[i]]==i

In [46]:
print('einit prob :',pi_hat)
print('eTransfor Maxtrix : ',A_hat)
print('eMean : ',means_hat)
print('ecovs : ',covars_hat)

einit prob : [ 0.  0.  1.  0.  0.]
eTransfor Maxtrix :  [[ 0.24444444  0.          0.43333333  0.32222222  0.        ]
 [ 0.          0.36082474  0.          0.60824742  0.03092784]
 [ 0.03406326  0.          0.47688564  0.          0.48905109]
 [ 0.43902439  0.27642276  0.          0.28455285  0.        ]
 [ 0.          0.10071942  0.6294964   0.          0.26978417]]
eMean :  [[ 30.0639231   30.22329147]
 [ -0.60371631  49.96859881]
 [-25.03026589  30.10629378]
 [-14.83607421   0.05082308]
 [ 14.94392893  -0.11255187]]
ecovs :  [[[ 2.64868455  0.47269017]
  [ 0.47269017  8.5150928 ]]

 [[ 4.09533483  0.03786587]
  [ 0.03786587  4.48396995]]

 [[ 0.18351585  0.02577099]
  [ 0.02577099  5.45292363]]

 [[ 6.25312678 -0.49577678]
  [-0.49577678  6.21899877]]

 [[ 7.86488426 -0.16833308]
  [-0.16833308  6.31717184]]]


In [48]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [49]:
plt.scatter(sample[:, 0], sample[:, 1], s=50, c=labels, cmap=plt.cm.Spectral, marker='o',
            label=u'观测值', linewidths=0.5, zorder=20)
plt.plot(sample[:, 0], sample[:, 1], 'r-', zorder=10)
plt.scatter(means[:, 0], means[:, 1], s=100, c=np.random.rand(n), marker='D', label=u'中心', alpha=0.8, zorder=30)
x1_min, x1_max = sample[:, 0].min(), sample[:, 0].max()
x2_min, x2_max = sample[:, 1].min(), sample[:, 1].max()
x1_min, x1_max = expand(x1_min, x1_max)
x2_min, x2_max = expand(x2_min, x2_max)
plt.xlim((x1_min, x1_max))
plt.ylim((x2_min, x2_max))
plt.legend(loc='upper left')
plt.grid(True)
plt.show()